May put this in same file as APW eventually

In [1]:
import numpy as np
from scipy import special as sp
import matplotlib.pyplot as plt

In [2]:
#constants
a = 6.83087
vol = 3*(a**3)/4

#simulation parameters
l_max = 5
nLim = 2

#recipricol lattice vectors 
b1 = 2*np.pi*np.array([-1,1,1])/a
b2 = 2*np.pi*np.array([1,-1,1])/a
b3 = 2*np.pi*np.array([1,1,-1])/a
Ks =[l*b1 + m*b2 + n*b3 for l in range(-nLim,nLim+1) for m in range(-nLim,nLim+1) for n in range(-nLim,nLim+1)]



In [3]:
#data for potential

rsAll = np.array([0,0.005,0.010,0.015,0.020,0.025,0.030,0.035,0.040,
              0.050,0.060,0.070,0.080,0.090,0.100,
              0.120,0.140,0.160,0.180,0.200,0.220,0.240,0.260,0.280,0.300,
              0.340,0.380,0.420,0.460,0.500,0.540,0.580,0.620,0.660,0.700
              ,0.800,0.900,1.00,1.10,1.20, 1.30, 1.40,
       1.60, 2.00, 2.40, 2.80])

R = rsAll[-1]

twoZ = np.array([58.000
,57.500,56.206,54.966,53.797,52.688,51.644,50.656,49.720,47.970,46.358,
        44.849,43.420,42.060,40.755,38.333,36.140,34.157,32.359,30.741,29.276,27.917,
        26.662,25.462,24.264,22.186,20.470,18.851,17.308,15.965,
        14.772,13.625,12.631,11.680,10.874,9.099,7.782,
        6.749,5.963,5.310,4.778,4.284,3.533,
        2.724,2.277,2.629])
Vs = -twoZ/(2*rsAll) #factor of 2 is due to units


C:\Users\willi\AppData\Local\Temp\ipykernel_21188\2533544679.py:17: RuntimeWarning: divide by zero encountered in divide
  Vs = -twoZ/(2*rsAll) #factor of 2 is due to units


In [4]:
#code for solution to the muffin part

def w(x,t,h,f):
    """
    w function in Numerov's method.
    """
    if x == 0:
        return 0
    return (1 - f(t)*(h**2)/12)*x

def wInv(w,t,h,f):
    """
    Inverse w function in Numerov's method.
    """
    return w/(1 - f(t)*(h**2)/12)


def getRatio(l,E):
    """
    Finds ratio required for the matrix calculation.
    """
    def f(r):
        """
        Finds f function for Numerov algorithm.
        """
        if r == 0.6:
            return l*(l+1)/r**2 - 2*(E + 21.88)
        index = list(rsAll).index(r)
        return l*(l+1)/r**2 + 2*(Vs[index] - E)

    def getUs(rs,us,h,iLen):
        """
        Runs iLen iterations of Numverov algorithm for rs,us lists (us contains only intial conditions).
        """
        for i in range(2,iLen+2):
            w_t = w(us[-1],rs[i-1],h,f)
            w_tMinush = w(us[-2],rs[i-2],h,f)
            w_rPlush = 2*w_t - w_tMinush + us[i-1]*f(rs[i-1])*(h**2)
            us.append(wInv(w_rPlush,rs[i],h,f))
        return us
    
    #runs numerov algorithm for the different values of r (different sections of lists have different size h's)
    us1 = getUs(rsAll[:9],[rsAll[0]**(l+1),rsAll[1]**(l+1)],0.005,7)

    rs2 = np.insert(rsAll[9:15],0,[rsAll[6],rsAll[8]])
    us2 = getUs(rs2,[us1[-3],us1[-1]],0.01,6)

    rs3 = np.insert(rsAll[15:25],0,[rsAll[12],rsAll[14]])
    us3 = getUs(rs3,[us2[-3],us2[-1]],0.02,10)

    rs4 = np.insert(rsAll[25:35],0,[rsAll[22],rsAll[24]])
    us4 = getUs(rs4,[us3[-3],us3[-1]],0.04,10)
    
    uAv = (us4[-4] + us4[-3])/2
    rs5 = np.insert(rsAll[35:42],0,[0.6,rsAll[34]])
    us5 = getUs(rs5,[uAv,us4[-1]],0.1,7)

    rs6 = np.insert(rsAll[42:46],0,[rsAll[37],rsAll[41]])
    us6 = getUs(rs6,[us5[-5],us5[-1]],0.4,4)

    #returns the required ratio
    uDotEnd = (us6[-1] - us6[-3])/(2*0.4)
    return uDotEnd/us6[-2] - 1/rsAll[-2] 

In [ ]:
def a(l,q):
        qMag = np.linalg.norm(q)
        return None
    
def b(l,q):
    qMag = np.linalg.norm(q) #should these stay inside the loop?
    return None

def U(K,l):
    KMag = np.linalg.norm(K)
    if KMag == 0:
        return 1 - 4*np.pi*(R**3)/(3*vol)
    return -4*np.pi*(R**2)*sp.spherical_jn(l,K*R)/K

def s(q1,q2,l,N_l):
    return a(l,q1)*a(l,q2) - b(l,q1)*b(l,q2)*N_l

In [ ]:


def getE(k):
    #run simulations


    #find normalisation conditions
    N = None #list/array of l's


    #calculate matricies
    qs = [k + K for K in Ks]

    

    
    #find overlap matrix
    S = np.empty([len(Ks),len(Ks)])

    for i,q_i in enumerate(qs):
        for j,q_j  in enumerate(qs):
            cosThetaij = np.dot(q_i,q_j)/(np.linalg.norm(q_i)*np.linalg.norm(q_j))
            S[i,j] = U(q_i - q_j)
            for l in range(l_max):
                S[i,j] += 4*np.pi*(R**4)*(2*l + 1)*sp.eval_legendre(l,cosThetaij)*s(q_i,q_j,l,N[l])/vol



    